In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
root_path = '/workdir/radish/PET-CT/3D_reggan/train/'


In [12]:
CHUNK_LENGTH = 5 
STEP = 4 
def save(save_dir, root_path, max_records = 6000): 
    input_dir = save_dir + '/A'
    output_dir = save_dir + '/B'
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    index = 0 
    num_patients = 0
    for folder in os.listdir(root_path):
        a = np.load(root_path + folder + '/predicted_volume.npy', allow_pickle=True)
        b = np.load(root_path + folder + '/pet.npy', allow_pickle=True)
        if (a.shape != b.shape):
            print(folder)
            print(a.shape)
            print(b.shape)
            print('----------------------')
            continue
        num_patients = num_patients + 1
        for i in tqdm(range(0, a.shape[0]- CHUNK_LENGTH, STEP)):
            inp = a[i:i+CHUNK_LENGTH]
            out = b[i:i+CHUNK_LENGTH] 
            np.save(f'{input_dir}/{index}.npy', inp)
            np.save(f'{output_dir}/{index}.npy', out)
            index = index + 1 
            if (index >= max_records):
                return (index, num_patients)

In [13]:
root_path = '/workdir/radish/PET-CT/3D_reggan/train/'
save_dir = '3d_model_data_5/train'
index, num =  save(save_dir, root_path, max_records = 7500)
print(f'Number of patients: {num}')

 53%|█████▎    | 39/74 [00:00<00:00, 1003.55it/s]

Number of patients: 109


In [15]:
root_path = '/workdir/radish/PET-CT/3D_reggan/val/'
save_dir = '3d_model_data_5/val'
index, num =  save(save_dir, root_path, max_records = 500)
print(f'Number of patients: {num}')

 74%|███████▍  | 55/74 [00:00<00:00, 758.84it/s]

Number of patients: 7


In [16]:
a = np.load ('/home/huutien/3D_PET-CT/3d_model_data_5/train/A/0.npy')
# b = np.load ('/home/huutien/3D_PET-CT/3d_model_data/train/B/0.npy')

In [17]:
a.shape

(5, 256, 256)

In [2]:
import math

CHUNK_DEPTH = 7
D, H, W = 300, 256, 256  # Assuming voxel shape is (D, 256, 256)

# Calculate the required number of chunks
num_chunks = math.ceil(D / CHUNK_DEPTH)

# Calculate overlap to ensure we fully cover D with equal-sized chunks (CHUNK_DEPTH)
if num_chunks > 1:
    OVERLAP = int((num_chunks * CHUNK_DEPTH - D) / (num_chunks - 1))
else:
    OVERLAP = 0  # No overlap needed if only one chunk is required

print(f"Calculated OVERLAP: {OVERLAP}")

Calculated OVERLAP: 0


In [5]:
OVERLAP = CHUNK_DEPTH -  (math.ceil(D / CHUNK_DEPTH) * CHUNK_DEPTH - D)

In [6]:
OVERLAP

6

In [1]:
import sys
import os
import torch
from tqdm import tqdm

experiment_name = "Experiment_ResidualUNet3D_lr1e-4"
N_EPOCHS = 100
LOG_DIR = f'./logs/{experiment_name}'
CHECKPOINT_DIR = LOG_DIR + '/checkpoints'
CHECKPOINT_PATH = CHECKPOINT_DIR + '/model_epoch_29.pth'  # Update to the latest checkpoint if needed

from model.unet3d import ResidualUNet3D, ResidualUNetSE3D

# Initialize model
IN_CHANNELS, OUT_CHANNELS = 1, 1
model = ResidualUNetSE3D(IN_CHANNELS, OUT_CHANNELS).to('cuda:1')
# read 3D data 
import numpy as np

file_path = '/workdir/radish/PET-CT/3D_reggan/dunganh/1.2.840.113619.2.55.3.663376.78.1704246084.428/predicted_volume.npy'

# Load checkpoint if available
if os.path.isfile(CHECKPOINT_PATH):
    print(f"Loading checkpoint from {CHECKPOINT_PATH}")
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint['epoch']  # Resume from the saved epoch
    print(f"Resuming from epoch {start_epoch}")

model = model.to('cuda:1')

Loading checkpoint from ./logs/Experiment_ResidualUNet3D_lr1e-4/checkpoints/model_epoch_29.pth


/tmp/ipykernel_2151/2582932746.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH)


Resuming from epoch 29


In [2]:
voxel = np.load(file_path, allow_pickle=True)

In [3]:
import torch
import numpy as np

# Define the depth of each chunk and overlap
CHUNK_DEPTH = 7
OVERLAP = 3  # Overlapping depth for each chunk
D, H, W = voxel.shape  # Assuming voxel shape is (D, 256, 256)

# Ensure voxel is on GPU
voxel = torch.tensor(voxel, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to('cuda:1')  # Shape: (1, 1, D, 256, 256)
voxel = voxel / 32767.0 
voxel =  (voxel - 0.5 ) * 2 

# Prepare tensors to store accumulated output and weights
output_accum = torch.zeros((1, 1, D, H, W), dtype=torch.float32).to('cuda:1')  # Accumulated output
weight_accum = torch.zeros((1, 1, D, H, W), dtype=torch.float32).to('cuda:1')  # Weight mask

# Process the voxel in chunks along the depth dimension
start_idx = 0
while start_idx < D:
    print(f"Processing chunk starting at index {start_idx}...")
    # Calculate end index for the chunk, with overlap
    end_idx = min(start_idx + CHUNK_DEPTH, D)
    
    # If this is the last chunk and smaller than CHUNK_DEPTH, adjust start_idx for overlap
    tmp = start_idx
    if end_idx - start_idx < CHUNK_DEPTH and start_idx != 0:
        start_idx = max(0, end_idx - CHUNK_DEPTH)  # Adjust start_idx to include overlap with the previous chunk
    
    # Select the chunk from the voxel
    chunk = voxel[:, :, start_idx:end_idx, :, :]  # Shape: (1, 1, CHUNK_DEPTH, 256, 256) or smaller if at the last chunk

    # Pass the chunk through the model
    with torch.no_grad():
        output_chunk = model(chunk)

    # Determine the slice in the full output corresponding to this chunk
    output_start = start_idx
    output_end = min(start_idx + CHUNK_DEPTH, D)
    
    # Add the model's output to the accumulated tensor
    output_accum[:, :, output_start:output_end, :, :] += output_chunk[:, :, :output_end - output_start, :, :]
    weight_accum[:, :, output_start:output_end, :, :] += 1  # Increment weights for overlapping regions

    # Move to the next chunk position, taking the overlap into account
    # print(start_idx, CHUNK_DEPTH, OVERLAP)
    start_idx = tmp
    start_idx = start_idx + CHUNK_DEPTH - OVERLAP
    # print(start_idx)

# Normalize the accumulated output by the weight mask to compute the average for overlapping regions
output_voxel = output_accum / weight_accum
# output_voxel = output_voxel.squeeze().cpu().numpy()  # Convert back to numpy if needed
output_voxel = output_voxel.mul(0.5).add(0.5).clamp(0, 1)
output_voxel = output_voxel * 32767.0
output_voxel = output_voxel.squeeze().cpu().numpy() 

Processing chunk starting at index 0...
Processing chunk starting at index 4...
Processing chunk starting at index 8...
Processing chunk starting at index 12...
Processing chunk starting at index 16...
Processing chunk starting at index 20...
Processing chunk starting at index 24...
Processing chunk starting at index 28...
Processing chunk starting at index 32...
Processing chunk starting at index 36...
Processing chunk starting at index 40...
Processing chunk starting at index 44...
Processing chunk starting at index 48...
Processing chunk starting at index 52...
Processing chunk starting at index 56...
Processing chunk starting at index 60...
Processing chunk starting at index 64...
Processing chunk starting at index 68...
Processing chunk starting at index 72...
Processing chunk starting at index 76...
Processing chunk starting at index 80...
Processing chunk starting at index 84...
Processing chunk starting at index 88...
Processing chunk starting at index 92...
Processing chunk st

In [4]:
output_voxel.min(), output_voxel.max(), output_voxel.shape

(0.0, 29276.516, (299, 256, 256))

In [5]:
# output_voxel = output_voxel.mul(0.5).add(0.5).clamp(0, 1)
# output_voxel = output_voxel * 32767.0
# output_voxel = output_voxel.squeeze().cpu().numpy() 

print(output_voxel.shape, voxel.shape, output_voxel.min(), output_voxel.max())
# Save the output voxel
np.save('/workdir/radish/PET-CT/3D_reggan/dunganh/1.2.840.113619.2.55.3.663376.78.1704246084.428/3D_model_overlap.npy', output_voxel)

(299, 256, 256) torch.Size([1, 1, 299, 256, 256]) 0.0 29276.516


In [9]:
# arg max of weight_accum[0,0, :, 0, 0 ]
tmp = weight_accum[0,0, :, 0, 0 ]
tmp

tensor([1., 1., 1., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2.,
        2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1.,
        2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2.,
        2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1.,
        2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2.,
        2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1.,
        2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2.,
        2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1.,
        2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2.,
        2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1.,
        2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2.,
        2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1.,
        2., 2., 2., 1., 2., 2., 2., 1., 

In [5]:
i = 1 
for vox in output_chunks: 
    print(vox.shape, i )
    i = i + 1

torch.Size([1, 1, 7, 256, 256]) 1
torch.Size([1, 1, 3, 256, 256]) 2
torch.Size([1, 1, 6, 256, 256]) 3
torch.Size([1, 1, 2, 256, 256]) 4
torch.Size([1, 1, 5, 256, 256]) 5
torch.Size([1, 1, 1, 256, 256]) 6
torch.Size([1, 1, 4, 256, 256]) 7
torch.Size([1, 1, 7, 256, 256]) 8
torch.Size([1, 1, 3, 256, 256]) 9
torch.Size([1, 1, 6, 256, 256]) 10
torch.Size([1, 1, 2, 256, 256]) 11
torch.Size([1, 1, 5, 256, 256]) 12
torch.Size([1, 1, 1, 256, 256]) 13
torch.Size([1, 1, 4, 256, 256]) 14
torch.Size([1, 1, 7, 256, 256]) 15
torch.Size([1, 1, 3, 256, 256]) 16
torch.Size([1, 1, 6, 256, 256]) 17
torch.Size([1, 1, 2, 256, 256]) 18
torch.Size([1, 1, 5, 256, 256]) 19
torch.Size([1, 1, 1, 256, 256]) 20
torch.Size([1, 1, 4, 256, 256]) 21
torch.Size([1, 1, 7, 256, 256]) 22
torch.Size([1, 1, 3, 256, 256]) 23
torch.Size([1, 1, 6, 256, 256]) 24
torch.Size([1, 1, 2, 256, 256]) 25
torch.Size([1, 1, 5, 256, 256]) 26
torch.Size([1, 1, 1, 256, 256]) 27
torch.Size([1, 1, 4, 256, 256]) 28
torch.Size([1, 1, 7, 256, 256

In [1]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 1000

# Generate features
age = np.random.randint(18, 66, size=n_samples)  # Age 18-65
income = np.random.randint(1000, 10001, size=n_samples)  # Income 1000-10000 USD
gender = np.random.choice([0, 1], size=n_samples)  # Gender: 0 = Male, 1 = Female
browsing_time = np.random.uniform(1, 120, size=n_samples)  # Browsing time in minutes
product_rating = np.random.uniform(1, 5, size=n_samples)  # Product rating 1-5

# Generate target variable (Buy_Product) with some randomness
# Customers with high income, high browsing time, and high product ratings are more likely to buy
buy_probability = (
    0.3 * (income / 10000) +
    0.2 * (browsing_time / 120) +
    0.3 * (product_rating / 5) +
    0.2 * (gender) +
    np.random.normal(0, 0.1, n_samples)
)

# Convert probabilities to binary classification (threshold = 0.5)
buy_product = (buy_probability > 0.5).astype(int)

# Create a DataFrame
data = pd.DataFrame({
    'Age': age,
    'Income': income,
    'Gender': gender,
    'Browsing_Time': browsing_time,
    'Product_Rating': product_rating,
    'Buy_Product': buy_product
})

# Save the dataset to a CSV file
output_file = 'classification_data.csv'
data.to_csv(output_file, index=False)

print(f"Classification dataset saved to {output_file}")

Classification dataset saved to classification_data.csv
